In [4]:
import numpy as np
import pandas as pd
import yfinance as yf

Backend
1) Import data
2) calculation 

Frontend
Info stat:
1) Start date
2) End date
3) Duration
4) Initial Equity
5) Final Equity
6) Return, ratios...
7) Max drawdown
8) Longest drawdown period

Trade stat:
1) Win rate
2) Best trade
3) Average position size

Plot:
1) Equity curve
2) P&L plot
3) Stock price graph with notations
4) Volume

In [ ]:
def Import_Data_yf():


In [ ]:
class Initial_Config:
    def __init__(self, Initial_Equity = 10000):
        self.Initial_Equity = Initial_Equity

class Backtesting:
    def __init__(self, df, initial_principal, rf_rate, period):
        self.df = df
        self.rf_rate = rf_rate
        self.period = period #year

        self.long_list = []
        self.short_list = []
        self.profit_list = []
        self.monthly_return_list = []
        self.monthly_return_index = 0
        self.monthly_profit_list = []
        self.peak = initial_principal
        self.bottom = initial_principal



    def isLong(self, index, rows, df):
        return rows['Alpha_A'] == df.loc[index-4:index, 'Alpha_A'].max() and rows['Alpha_B'] == df.loc[index-4:index, 'Alpha_B'].max() and rows['Alpha_C'] == df.loc[index-4:index, 'Alpha_C'].max()

    def isShort(self, index, rows, df):
        return rows['Alpha_A'] == df.loc[index-4:index, 'Alpha_A'].min() and rows['Alpha_B'] == df.loc[index-4:index, 'Alpha_B'].min() and rows['Alpha_C'] == df.loc[index-4:index, 'Alpha_C'].min()

    def long(self, rows, df): 
        date = rows['timestamp']
        self.long_date.append(date)
        excute_price = df[df['timestamp'] == date]['close'].values[0]
        if self.short_list:
            profit = self.short_list[0] - excute_price
            self.profit_list.append(profit)
            self.principal += profit
            self.peak = max(self.peak, self.principal)
            self.bottom = min(self.bottom, self.principal)
            drawdown = (self.peak - self.bottom) / self.peak
            self.max_drawdown = max(self.max_drawdown, drawdown)
            self.short_list.pop(0)
        else:
            self.long_list.append(excute_price)

    def short(self, rows, df):
        date = rows['timestamp']
        self.short_date.append(date)
        excute_price = df[df['timestamp'] == date]['close'].values[0]
        if self.long_list:
            profit = excute_price - self.long_list[0]
            self.profit_list.append(profit)
            self.principal += profit
            self.peak = max(self.peak, self.principal)
            self.bottom = min(self.bottom, self.principal)
            drawdown = self.peak - self.bottom
            self.max_drawdown = max(self.max_drawdown, drawdown)
            self.long_list.pop(0)
        else:
            self.short_list.append(excute_price)

    def monthly_update(self, rows):
        if (rows['timestamp'][-14:] in self.month_end_hour):
            monthly_profit = float(np.sum(self.profit_list[self.monthly_return_index:]))
            monthly_return = monthly_profit/self.principal 
            self.monthly_profit_list.append(monthly_profit)
            self.monthly_return_list.append(monthly_return)
            self.monthly_return_index = len(self.profit_list)

    def close_position(self, rows, df):
        if self.long_list:
            for i in self.short_list:
                self.short(rows, df)
        if self.short_list:
            for i in self.long_list:
                self.long(rows, df)

    def setsharpe(self):
        monthly_return_stddev = np.std(self.monthly_return_list, ddof=0)
        average_monthly_return = np.average(self.monthly_return_list)
        sharpe = (average_monthly_return - self.rf_rate) / monthly_return_stddev
        self.sharpe = sharpe
    
    def setCAGR(self):
        self.CAGR = np.power((self.principal/self.initial_principal), (1/self.period)) - 1

    def setcalmarratio(self):
        total_return = 1
        for i in self.monthly_return_list:
            total_return *= (1 + i)
        annualized_return = np.power(total_return, (1/self.period)) - 1
        print(annualized_return)
        self.calmarratio = annualized_return / self.max_drawdown

    def plotequitycurve(self):
        try:
            plt.plot(self.df['timestamp'], self.df['principal'], label = 'Equity curve')
            plt.xlabel('Time')
            plt.ylabel('Principal (million)')
            plt.legend()
            plt.show()
        except:
            print("exception in plotting equity curve")
        